In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install "datasets>=2.5.2" "numpy>=1.24.2" "pandas>=1.4.4" "scikit-learn>=1.1.1" "torch>=1.10.2" "tqdm>=4.64.1" "transformers>=4.22.1"


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Tabula-main-claude

/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude


In [3]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula import Tabula
import pandas as pd

In [4]:

import pandas as pd
from sklearn.model_selection import train_test_split

original_data = pd.read_csv("Real_Datasets/Adult.csv")

# train_data, test_data = train_test_split(original_data, train_size=40000, random_state=42)

# train_data.to_csv("Real_Datasets/adult_train.csv", index=False)
# test_data.to_csv("Real_Datasets/adult_test.csv", index=False)

train_data = pd.read_csv("Real_Datasets/adult_train.csv")


# train_data.to_csv("/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/Real_Datasets/adult_train.csv", index=False)
# test_data.to_csv("/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/Real_Datasets/adult_test.csv", index=False)

# train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/Real_Datasets/Insurance_train.csv")

In [5]:
train_data

,age,workclass,fnlwgt,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,31,4,208785,1,4,3,1,4,1,0,0,30,39,0
1,20,4,245182,15,4,1,3,4,0,0,0,10,39,0
2,41,4,100451,11,2,3,0,4,1,7688,0,38,39,1
3,24,4,243368,13,4,5,1,4,1,0,0,36,26,0
4,47,4,159399,14,2,10,0,4,1,0,0,55,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,36,4,635913,11,3,8,1,2,1,0,0,40,39,0
39996,34,4,107624,15,2,3,0,4,1,0,0,50,39,0
39997,28,4,250135,15,0,4,1,4,0,0,0,40,39,0
39998,46,7,96652,8,5,1,4,2,0,0,0,40,39,0


In [6]:
import torch
categorical_columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country", "income"]
for sigma_value in [0.02, 0.03, 0.04]:  
    model = Tabula(
        llm='distilgpt2',
        experiment_dir="adult_training",
        batch_size=64,
        epochs=8,
        categorical_columns=categorical_columns,
        use_dp=True,
        clip_coeff=1,
        sigma=sigma_value,
        micro_batch_size=16,
        learning_rate=5e-4
    )


    model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)
    print(f"Now, it's training under sigma {sigma_value}")
    model.fit(train_data)

    torch.save(model.model.state_dict(), "insurance_training/model_400epoch.pt")

    synthetic_data = model.sample(n_samples=40000)
    sigma_str = f"{sigma_value:.2f}".replace('.', '')
    synthetic_data.to_csv(f"adult_syn_data/adult-sigma-{sigma_str}.csv", index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

40045it [04:35, 145.24it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
<ipython-input-6-9a07adbf99ee>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the 

Now, it's training under sigma 0.04


/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula_trainer.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.970100
1000,0.972500
1500,0.981000
2000,0.985300
2500,0.984800
3000,0.982800
3500,0.978600
4000,0.973500
4500,0.970000
5000,0.966600


40006it [04:53, 136.43it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')


In [6]:
import torch
categorical_columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country", "income"]
for sigma_value in [0.01, 0.05, 0.1, 0.15]:  
    model = Tabula(
        llm='distilgpt2',
        experiment_dir="adult_training",
        batch_size=64,
        epochs=8,
        categorical_columns=categorical_columns,
        use_dp=True,
        clip_coeff=1,
        sigma=sigma_value,
        micro_batch_size=16,
        learning_rate=5e-4
    )


    model.model.load_state_dict(torch.load("pretrained-model/model.pt"), strict=False)
    print(f"Now, it's training under sigma {sigma_value}")
    model.fit(train_data)

    torch.save(model.model.state_dict(), "insurance_training/model_400epoch.pt")

    synthetic_data = model.sample(n_samples=40000)
    sigma_str = f"{sigma_value:.2f}".replace('.', '')
    synthetic_data.to_csv(f"adult_syn_data/adult-sigma-{sigma_str}.csv", index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

40033it [06:57, 95.99it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
<ipython-input-6-f86c1810b9f1>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the f

Now, it's training under sigma 0.15


/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula_trainer.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TabulaTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.048800
1000,1.070200
1500,1.094900
2000,1.104000
2500,1.105000
3000,1.100200
3500,1.092400
4000,1.083400
4500,1.075900
5000,1.070100


40008it [09:23, 71.04it/s]
/content/drive/MyDrive/Colab Notebooks/Tabula-main-claude/tabula/tabula.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
